# Upgrade OCR
## Introduction
The goal of this file is to find preprocessing ways in order to upgrade OCR's efficiency

## Import

In [ ]:
import os
from time import time
import pathlib
import matplotlib.pyplot as plt
import cv2
from models_pipeline import easyOCR
from utils_.functions import plot_confusion_matrix, rotate_image
from cv2 import dnn_Model

In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as im
from scipy.ndimage import interpolation as inter

## Constant

In [ ]:
IMAGES_PATH = './utils_/data-examples/ocr/'
OUTPUT_PATH = './outputs/'
JSON_TEST_PATH = './utils_/data-examples/ocr/references.json'

## Code

In [ ]:
def plot_images(im_before, im_after):
    fig, axes = plt.subplots(nrows=1, ncols=2, dpi=600)
    fig.tight_layout()
    
    axes[0].imshow(im_before, cmap="gray")
    axes[0].set_title("Before")
    axes[0].axis("off")
    
    axes[1].imshow(im_after, cmap="gray")
    axes[1].set_title("After")
    axes[1].axis("off")
    
    plt.show()

In [ ]:
def plot_images_couples(images_be, images_af):
    if len(images_be) != len(images_af):
        raise Exception('Not the same length between images_be and images_af')
    fig, axes = plt.subplots(nrows=len(images_be), ncols=2, dpi=600)
    fig.tight_layout
    
    axes[0][0].set_title('Before')
    axes[0][1].set_title('After')
    
    for row,ax in enumerate(axes):
        ax[0].imshow(images_be[row], cmap="gray")
        ax[0].axis("off")
        
        ax[1].imshow(images_af[row], cmap="gray")
        ax[1].axis("off")


In [ ]:
pathlib.Path(OUTPUT_PATH).mkdir(parents=True, exist_ok=True)

In [ ]:
folder = os.listdir(IMAGES_PATH)
images = []
for file in folder:
    if file.endswith(".jpg"):
        images.append(file)
        
print(images)

In [ ]:
ocr = easyOCR()
config = {
    "low_text":0.5,
    "threshold":0.5,
    "min_size":5,
    "mag_ratio":3,
    "paragraph":True,
    "detail":1,
    "bbox_min_size":1,
    "contrast_ths":0.3,
    "adjust_contrast":0.5,
    "rotation_info":[180]
}

In [ ]:
bleu, process_time, n_sentences = ocr.test(JSON_TEST_PATH, IMAGES_PATH, True, config)

## Preprocessing

In [ ]:
image = cv2.imread(os.path.join(IMAGES_PATH+images[0]), cv2.IMREAD_GRAYSCALE)

txt_fo = ocr.predict(image=image, **config)
print(txt_fo)

https://towardsdatascience.com/pre-processing-in-ocr-fc231c6035a7

#### *Adaptative Binarization*

In [ ]:
imgf = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2) #imgf contains Binary image

plot_images(image, imgf)

#### *Rotation*

In [ ]:
img = cv2.imread(os.path.join(IMAGES_PATH, '3.png'), cv2.IMREAD_GRAYSCALE)

def find_score(arr, angle):
    data = inter.rotate(arr, angle, reshape=False, order=0)
    hist = np.sum(data, axis=1)
    score = np.sum((hist[1:] - hist[:-1]) ** 2)
    return hist, score
delta = 7
limit = 100
angles = np.arange(-limit, limit+delta, delta)
scores = []

for angle in angles:
    hist, score = find_score(img, angle)
    scores.append(score)
    
best_score = max(scores)
best_angle = angles[scores.index(best_score)]
print('Best angle: {}'.format(best_angle))# correct skew
data = inter.rotate(img, best_angle, reshape=True, order=0)

plot_images(img, data)

#### *Denoizer*

In [ ]:
# Reading image from folder where it is stored 
img = cv2.imread(os.path.join(IMAGES_PATH+images[0]), cv2.IMREAD_COLOR)
# denoising of image saving it into dst image 
dst = cv2.fastNlMeansDenoisingColored(img, None, 5, 10, 4, 15) 
# Plotting of source and destination image 

plot_images(img,dst)
print(txt_fo)
print(ocr.predict(image=dst, **config))

#### *Denoizer + Adaptative Threshold*

In [ ]:
dst = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
imgf = cv2.adaptiveThreshold(dst,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,7,2) #imgf contains Binary image

plot_images(img, imgf)
print(txt_fo)
print(ocr.predict(image=imgf, **config))

#### *Erosion*

In [ ]:
kernel = np.ones((2,2),np.uint8)
gray_negative = abs(255-imgf)
erosion = cv2.erode(gray_negative,kernel,iterations = 1)

plot_images(img, abs(255-erosion))
print(ocr.predict(image=abs(255-erosion), **config))

## OCR Detection

In [ ]:
def draw_bboxes(bboxes, img: np.ndarray):
    img_returned = img.copy()
    for bbox in bboxes:
        a,b,c,d = bbox
        cv2.rectangle(img_returned, (a, d), (b, c), (0, 255, 0))
    return img_returned

In [ ]:
def extract_bboxes(bboxes, img: np.ndarray, m=0):
    img_returned = []
    for bbox in bboxes:
        a,b,c,d = bbox
        print(bbox)
        img_returned.append(img[c-m:d+m,a-m:b+m])
    return img_returned
    

In [ ]:
ocr = easyOCR()

res = ocr.model.detect(
    img=img,
    min_size=7,
    low_text=0.5
    )   

In [ ]:
img_lst = []
new_img_lst = []
for image_fn in images:
    im = cv2.imread(os.path.join(IMAGES_PATH, image_fn), cv2.IMREAD_GRAYSCALE)
    img_lst.append(im)
    
    res = ocr.model.detect(
        img=im,
        min_size=7,
        low_text=0.5
    )   
    new_img_lst.append(draw_bboxes(res[0][0], im))

plot_images_couples(img_lst, new_img_lst)

In [ ]:
im = cv2.imread(os.path.join(IMAGES_PATH, images[-1]), cv2.IMREAD_GRAYSCALE)

res = ocr.model.detect(
    img=im,
    min_size=7,
    low_text=0.5
)
delivery_address_boxes = res[0][0][:2]
img_bboxes = extract_bboxes(delivery_address_boxes, im)
plot_images(im,img_bboxes[1])



In [ ]:
imgf = cv2.adaptiveThreshold(img_bboxes[1],255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,7, 2) #imgf contains Binary image

plot_images(img_bboxes[1], imgf)
print(ocr.predict(image=imgf))

In [ ]:
kernel = np.ones((1,2),np.uint8)
gray_negative = abs(255-imgf)
erosion = cv2.erode(gray_negative,kernel,iterations = 1)

plot_images(img, abs(255-erosion))
print(ocr.predict(image=abs(255-erosion)))

In [ ]:
print(ocr.predict(image=img_bboxes[1]))
print(ocr.predict(image=im, **config))

In [ ]:
kernel = np.ones((2,2),np.uint8)
gray_negative = abs(255-img_bboxes[1])
erosion = cv2.erode(gray_negative,kernel,iterations = 1)

plot_images(img_bboxes[1], abs(255-erosion))
print(ocr.predict(image=abs(255-erosion)))

#### *Downsizing interpolation*

In [ ]:
resized = cv2.resize(img, (int(img.shape[1]/2),int(img.shape[0]/2)), interpolation=cv2.INTER_LINEAR)

plot_images(img, resized)

## Super resolution

In [ ]:
import torch
import torch_enhance

# increase resolution by factor of 2 (e.g. 128x128 -> 256x256)
model = torch_enhance.models.SRResNet(scale_factor=2, channels=1)

In [ ]:
print(model)

In [ ]:
im_lr = img_bboxes[1].copy()
im_lr = im_lr.reshape(1, *im_lr.shape, 1)

lr = torch.tensor(im_lr, dtype=torch.float32)
lr = lr.permute((0,3,1,2))

sr = model(lr) # [1, 3, 256, 256]

img_sr = sr.permute(0,2,3,1).detach().numpy()

plot_images(im_lr[0], img_sr[0])